In [14]:
import matplotlib.pyplot as plt
from kinematics import kinematics
from ipywidgets import *

# make the plot larger
plt.rcParams['figure.dpi'] = 150

k = kinematics()

# output = k.leg_IK([0, 0, -0,3])


def update(roll, pitch, yaw, x, y, z, rot_x, rot_y, rot_z):
    k.plot_robot(xyz=[[x,y,z],[x,y,z],[x,y,z],[x,y,z]], rot=[roll,pitch,yaw], 
                 is_radians=False, center_offset=[rot_x,rot_y,rot_z])

# plot the robot with the sliders for adjusting the rotation and center_offset
interact(update,
         roll= widgets.FloatSlider(value=0, min=-20, max=20, step=1),
         pitch= widgets.FloatSlider(value=0, min=-20, max=20, step=1),
         yaw= widgets.FloatSlider(value=0, min=-20, max=20, step=1),
         x= widgets.FloatSlider(value=0, min=-0.10, max=0.10, step=0.01),
         y= widgets.FloatSlider(value=0, min=-0.10, max=0.10, step=0.01),
         z= widgets.FloatSlider(value=-0.10, min=-0.4, max=-0.10, step=0.01),
         rot_x= widgets.FloatSlider(value=0, min=-0.25205/2, max=0.25205/2, step=0.01),
         rot_y= widgets.FloatSlider(value=0, min=-0.105577/2, max=0.105577/2, step=0.01),
         rot_z= widgets.FloatSlider(value=0, min=-0.25, max=0.25, step=0.01));





interactive(children=(FloatSlider(value=0.0, description='roll', max=20.0, min=-20.0, step=1.0), FloatSlider(v…

In [33]:
import matplotlib.pyplot as plt
from kinematics import kinematics
from ipywidgets import *

# make the plot larger
plt.rcParams['figure.dpi'] = 150

k = kinematics()

# Define the update function
def update(roll, pitch, yaw, x, y, z, rot_x, rot_y, rot_z):
    foot_positions = [[x, y, z], [x, y, z], [x, y, z], [x, y, z]]
    
    # Plot the robot
    k.plot_robot(xyz=foot_positions, rot=[roll, pitch, yaw], 
                 is_radians=False, center_offset=[rot_x, rot_y, rot_z])
    
    # Calculate and print joint angles for each leg
    for i, foot_position in enumerate(foot_positions):
        angles = k.leg_IK(foot_position, rot=[roll, pitch, yaw], legID=i, is_radians=False, center_offset=[rot_x, rot_y, rot_z])
        print(f"Leg {i} angles: Hip: {angles[0]:.2f}, Thigh: {angles[1]:.2f}, Calf: {angles[2]:.2f}")
        #print angles in degree as well 
        #print(f"Leg {i} angles: Hip: {angles[0]*180/3.14159:.2f}, Thigh: {angles[1]*180/3.14159:.2f}, Calf: {angles[2]*180/3.14159:.2f}")

# Use interact to create sliders for adjusting the parameters
interact(update,
         roll=widgets.FloatSlider(value=0, min=-20, max=20, step=1),
         pitch=widgets.FloatSlider(value=0, min=-20, max=20, step=1),
         yaw=widgets.FloatSlider(value=0, min=-20, max=20, step=1),
         x=widgets.FloatSlider(value=0, min=-0.10, max=0.10, step=0.01),
         y=widgets.FloatSlider(value=0, min=-0.10, max=0.10, step=0.01),
         z=widgets.FloatSlider(value=-0.2, min=-0.4, max=-0.1, step=0.01),
         rot_x=widgets.FloatSlider(value=0, min=-0.25205/2, max=0.25205/2, step=0.01),
         
         rot_y=widgets.FloatSlider(value=0, min=-0.105577/2, max=0.105577/2, step=0.01),
         rot_z=widgets.FloatSlider(value=0, min=-0.25, max=0.25, step=0.01));



#Hip joint: -60 ~ 60 degrees, in radians: -1.0472 ~ 1.0472
#Thigh joint: -38 ~ 170 degrees, in radians: -0.663225115 ~ 2.96705973
#Calf joint: -156 ~ -48 degrees, in radians: -2.70526034 ~ -0.837758041



interactive(children=(FloatSlider(value=0.0, description='roll', max=20.0, min=-20.0, step=1.0), FloatSlider(v…

In [41]:
import numpy as np
from numpy.linalg import norm
from numpy import asarray, array
from math import atan2, acos, pi, sin, cos, radians

def point_to_rad(p1, p2):
    return atan2(p2, p1)

def RotMatrix3D(rotation=[0, 0, 0], is_radians=True, order='xyz'):
    roll, pitch, yaw = rotation

    if not is_radians:
        roll = radians(roll)
        pitch = radians(pitch)
        yaw = radians(yaw)

    rotX = np.array([[1, 0, 0], [0, cos(roll), -sin(roll)], [0, sin(roll), cos(roll)]])
    rotY = np.array([[cos(pitch), 0, sin(pitch)], [0, 1, 0], [-sin(pitch), 0, cos(pitch)]])
    rotZ = np.array([[cos(yaw), -sin(yaw), 0], [sin(yaw), cos(yaw), 0], [0, 0, 1]])

    if order == 'xyz':
        rotationMatrix = rotZ @ rotY @ rotX
    elif order == 'xzy':
        rotationMatrix = rotY @ rotZ @ rotX
    elif order == 'yxz':
        rotationMatrix = rotZ @ rotX @ rotY
    elif order == 'yzx':
        rotationMatrix = rotX @ rotZ @ rotY
    elif order == 'zxy':
        rotationMatrix = rotY @ rotX @ rotZ
    elif order == 'zyx':
        rotationMatrix = rotX @ rotY @ rotZ

    return rotationMatrix

class Kinematics:
    def __init__(self):
        self.right_legs = [2, 3]
        
        self.link_1 = 0.08
        self.link_2 = 0.213
        self.link_3 = 0.213
        self.phi = radians(90)
        
        self.length = 0.3762
        self.width = 0.0935
        
        self.leg_origins = np.array([
            [self.length / 2, self.width / 2, 0],
            [-self.length / 2, self.width / 2, 0],
            [-self.length / 2, -self.width / 2, 0],
            [self.length / 2, -self.width / 2, 0],
        ])

    def leg_IK(self, xyz, rot=[0, 0, 0], legID=0, is_radians=True, center_offset=[0, 0, 0]):
        is_right = legID in self.right_legs
        
        XYZ = asarray((np.linalg.inv(RotMatrix3D(rot, is_radians)) @
                      ((array(xyz) + self.leg_origins[legID] - array(center_offset)).transpose())).transpose())
        
        xyz_ = asarray(XYZ - self.leg_origins[legID] + array(center_offset)).flatten()
        return self.leg_IK_calc(xyz_, is_right)

    def leg_IK_calc(self, xyz, is_right=False):
        x, y, z = xyz[0], xyz[1], xyz[2]
        len_A = norm([0, y, z])
        
        a_1 = point_to_rad(y, z)
        a_2 = np.arcsin(np.sin(self.phi) * self.link_1 / len_A)
        a_3 = pi - a_2 - self.phi
        
        theta_1 = a_1 - a_3 if is_right else a_1 + a_3
        theta_1 = theta_1 % (2 * pi)

        j2 = array([0, self.link_1 * cos(theta_1), self.link_1 * sin(theta_1)])
        j4 = array(xyz)
        j4_2_vec = j4 - j2
        
        R = theta_1 - self.phi - pi / 2 if is_right else theta_1 + self.phi - pi / 2
        rot_mtx = RotMatrix3D([-R, 0, 0], is_radians=True)
        j4_2_vec_ = rot_mtx @ (np.reshape(j4_2_vec, [3, 1]))
        
        x_, y_, z_ = j4_2_vec_[0], j4_2_vec_[1], j4_2_vec_[2]
        len_B = norm([x_, z_])
        
        if len_B >= (self.link_2 + self.link_3):
            len_B = (self.link_2 + self.link_3) * 0.99999
            print(f'Target coordinate: [{x}, {y}, {z}] too far away')
        
        b_1 = point_to_rad(x_, z_)
    
        b_2 = acos((self.link_2**2 + len_B**2 - self.link_3**2) / (2 * self.link_2 * len_B))
        b_3 = acos((self.link_2**2 + self.link_3**2 - len_B**2) / (2 * self.link_2 * self.link_3))
        
        theta_2 = b_1 - b_2
        theta_3 = pi - b_3
        
        angles = self.angle_corrector([theta_1, theta_2, theta_3], is_right)
        return [angles[0], angles[1], angles[2]]

    def angle_corrector(self, angles=[0, 0, 0], is_right=True):
        angles[1] -= 1.5 * pi
        
        if is_right:
            theta_1 = angles[0] - pi
            theta_2 = angles[1] + 45 * pi / 180
        else:
            theta_1 = angles[0] - 2 * pi if angles[0] > pi else angles[0]
            theta_2 = -angles[1] - 45 * pi / 180
        
        theta_3 = -angles[2] + 45 * pi / 180
        return [theta_1, theta_2, theta_3]

# Example usage
kin = Kinematics()

# Assuming we want to find the joint angles for each leg with x, y = 0 and varying z
foot_positions = [
    [0, 0, -0.34],  # Front-left
    [0, 0, -0.34],  # Front-right
    [0, 0, -0.34],  # Rear-left
    [0, 0, -0.34],  # Rear-right
]

for i, foot_position in enumerate(foot_positions):

    angles = kin.leg_IK(foot_position, rot=[0, 0, 0], legID=i, is_radians=False, center_offset=[0, 0, 0])
    
    print(f"Leg {i} angles: Hip: {angles[0]}, Thigh: {angles[1]}, Calf: {angles[2]} ")

    #Print them in degrees
    
    print(f" \n Leg {i} angles: Hip: {angles[0] * 180 / pi}, Thigh: {angles[1] * 180 / pi}, Calf: {angles[2] * 180 / pi}")


Leg 0 angles: Hip: -0.23752117078144686, Thigh: 6.180737519358092, Calf: -0.5805025877544601 
 
 Leg 0 angles: Hip: -13.608960630782951, Thigh: 354.1301741373766, Calf: -33.26034827475326
Leg 1 angles: Hip: -0.23752117078144686, Thigh: 6.180737519358092, Calf: -0.5805025877544601 
 
 Leg 1 angles: Hip: -13.608960630782951, Thigh: 354.1301741373766, Calf: -33.26034827475326
Leg 2 angles: Hip: 0.2375211707814464, Thigh: -6.180737519358092, Calf: -0.5805025877544596 
 
 Leg 2 angles: Hip: 13.608960630782926, Thigh: -354.1301741373766, Calf: -33.26034827475324
Leg 3 angles: Hip: 0.2375211707814464, Thigh: -6.180737519358092, Calf: -0.5805025877544596 
 
 Leg 3 angles: Hip: 13.608960630782926, Thigh: -354.1301741373766, Calf: -33.26034827475324


In [12]:
import numpy as np
from numpy.linalg import norm
from numpy import asarray, array
from math import atan2, acos, pi, sin, cos, radians

def point_to_rad(p1, p2):
    return atan2(p2, p1)

def RotMatrix3D(rotation=[0, 0, 0], is_radians=True, order='xyz'):
    roll, pitch, yaw = rotation

    if not is_radians:
        roll = radians(roll)
        pitch = radians(pitch)
        yaw = radians(yaw)

    rotX = np.array([[1, 0, 0], [0, cos(roll), -sin(roll)], [0, sin(roll), cos(roll)]])
    rotY = np.array([[cos(pitch), 0, sin(pitch)], [0, 1, 0], [-sin(pitch), 0, cos(pitch)]])
    rotZ = np.array([[cos(yaw), -sin(yaw), 0], [sin(yaw), cos(yaw), 0], [0, 0, 1]])

    if order == 'xyz':
        rotationMatrix = rotZ @ rotY @ rotX
    elif order == 'xzy':
        rotationMatrix = rotY @ rotZ @ rotX
    elif order == 'yxz':
        rotationMatrix = rotZ @ rotX @ rotY
    elif order == 'yzx':
        rotationMatrix = rotX @ rotZ @ rotY
    elif order == 'zxy':
        rotationMatrix = rotY @ rotX @ rotZ
    elif order == 'zyx':
        rotationMatrix = rotX @ rotY @ rotZ

    return rotationMatrix

class Kinematics:
    def __init__(self):
        self.right_legs = [2, 3]
        
        self.link_1 = 0.08
        self.link_2 = 0.213
        self.link_3 = 0.213
        self.phi = radians(90)
    
        
        self.length = 0.3762
        self.width = 0.0935
        
        self.leg_origins = np.array([
            [self.length / 2, self.width / 2, 0],
            [-self.length / 2, self.width / 2, 0],
            [-self.length / 2, -self.width / 2, 0],
            [self.length / 2, -self.width / 2, 0],
        ])
        
        # Joint limits in radians
        self.hip_limits = (-1.0472, 1.0472)
        self.thigh_limits = (-0.663225115, 2.96705973)
        self.calf_limits = (-2.70526034, -0.837758041)

    def leg_IK(self, xyz, rot=[0, 0, 0], legID=0, is_radians=True, center_offset=[0, 0, 0]):
        is_right = legID in self.right_legs
        
        XYZ = asarray((np.linalg.inv(RotMatrix3D(rot, is_radians)) @
                      ((array(xyz) + self.leg_origins[legID] - array(center_offset)).transpose())).transpose())
        
        xyz_ = asarray(XYZ - self.leg_origins[legID] + array(center_offset)).flatten()
        return self.leg_IK_calc(xyz_, is_right)

    def leg_IK_calc(self, xyz, is_right=False):
        x, y, z = xyz[0], xyz[1], xyz[2]
        len_A = norm([0, y, z])
        
        a_1 = point_to_rad(y, z)
        a_2 = np.arcsin(np.sin(self.phi) * self.link_1 / len_A)
        a_3 = pi - a_2 - self.phi
        
        theta_1 = a_1 - a_3 if is_right else a_1 + a_3
        theta_1 = theta_1 % (2 * pi)

        j2 = array([0, self.link_1 * cos(theta_1), self.link_1 * sin(theta_1)])
        j4 = array(xyz)
        j4_2_vec = j4 - j2
        
        R = theta_1 - self.phi - pi / 2 if is_right else theta_1 + self.phi - pi / 2
        rot_mtx = RotMatrix3D([-R, 0, 0], is_radians=True)
        j4_2_vec_ = rot_mtx @ (np.reshape(j4_2_vec, [3, 1]))
        
        x_, y_, z_ = j4_2_vec_[0], j4_2_vec_[1], j4_2_vec_[2]
        len_B = norm([x_, z_])
        
        if len_B >= (self.link_2 + self.link_3):
            len_B = (self.link_2 + self.link_3) * 0.99999
            print(f'Target coordinate: [{x}, {y}, {z}] too far away')
        
        b_1 = point_to_rad(x_, z_)
        b_2 = acos((self.link_2**2 + len_B**2 - self.link_3**2) / (2 * self.link_2 * len_B))
        b_3 = acos((self.link_2**2 + self.link_3**2 - len_B**2) / (2 * self.link_2 * self.link_3))
        
        theta_2 = b_1 - b_2
        theta_3 = pi - b_3
        
        angles = self.angle_corrector([theta_1, theta_2, theta_3], is_right)
        angles = self.apply_joint_limits(angles)
        return [angles[0], angles[1], angles[2]]

    def angle_corrector(self, angles=[0, 0, 0], is_right=True):
        angles[1] -= 1.5 * pi
        
        if is_right:
            theta_1 = angles[0] - pi
            theta_2 = angles[1] + 45 * pi / 180
        else:
            theta_1 = angles[0] - 2 * pi if angles[0] > pi else angles[0]
            theta_2 = -angles[1] - 45 * pi / 180
        
        theta_3 = -angles[2] + 45 * pi / 180
        return [theta_1, theta_2, theta_3]
    
    def apply_joint_limits(self, angles):
        # Apply joint limits
        angles[0] = np.clip(angles[0], self.hip_limits[0], self.hip_limits[1])
        angles[1] = np.clip(angles[1], self.thigh_limits[0], self.thigh_limits[1])
        angles[2] = np.clip(angles[2], self.calf_limits[0], self.calf_limits[1])
        return angles

# Example usage
kin = Kinematics()

# Assuming we want to find the joint angles for each leg with x, y = 0 and varying z
foot_positions = [
    [0, 0, -0.3],  # Front-left
    [0, 0, -0.3],  # Front-right
    [0, 0, -0.3],  # Rear-left
    [0, 0, -0.3],  # Rear-right
]

for i, foot_position in enumerate(foot_positions):
    angles = kin.leg_IK(foot_position, legID=i)
    print(f"Leg {i} angles: Hip: {angles[0]}, Thigh: {angles[1]}, Calf: {angles[2]}")

    print(f" \n Leg {i} angles: Hip: {angles[0] * 180 / pi}, Thigh: {angles[1] * 180 / pi}, Calf: {angles[2] * 180 / pi}")
    


Leg 0 angles: Hip: -0.26993279583340346, Thigh: 2.96705973, Calf: -0.8641455751540232
 
 Leg 0 angles: Hip: -15.466009953420551, Thigh: 170.00000009222555, Calf: -49.51189434123062
Leg 1 angles: Hip: -0.26993279583340346, Thigh: 2.96705973, Calf: -0.8641455751540232
 
 Leg 1 angles: Hip: -15.466009953420551, Thigh: 170.00000009222555, Calf: -49.51189434123062
Leg 2 angles: Hip: 0.26993279583340346, Thigh: -0.663225115, Calf: -0.8641455751540232
 
 Leg 2 angles: Hip: 15.466009953420551, Thigh: -37.99999995657867, Calf: -49.51189434123062
Leg 3 angles: Hip: 0.26993279583340346, Thigh: -0.663225115, Calf: -0.8641455751540232
 
 Leg 3 angles: Hip: 15.466009953420551, Thigh: -37.99999995657867, Calf: -49.51189434123062
